<a href="https://colab.research.google.com/github/ssawant/TensorFlow-in-Practice/blob/main/C3W4_NLP_Reading_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os

print(tf.__version__)

2.4.0


In [2]:
file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(file, 'rb').read().decode(encoding='utf-8')

print(f"{len(text)}")
print(f"{text[:250]}")

1122304/1115394 [==============================] - 0s 0us/step
1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [3]:
vocab = sorted(set(text))
print(f"{len(vocab)}")
# print(f"{vocab}")

65


In [4]:
# creating a mapping for unique characters to indices
char2indx = {u:i for i, u in enumerate(vocab)}
indx2char = np.array(vocab)

text_as_int = np.array([char2indx[c] for c in text])
# print(f"{len(text_as_int)}")
print(f"{char2indx['C']}")
print(f"{text[:13]} ===> {text_as_int[:13]}")

15
First Citizen ===> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [5]:
seq_length = 100
example_per_epoch = len(text)

# Creating training example/ traget
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
  # print(i.numpy())
  print(indx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [6]:
sequence = char_dataset.batch(seq_length+1, drop_remainder=True)

print(sequence.take(13))

<TakeDataset shapes: (101,), types: tf.int64>


In [7]:
# return two list with first and last char removed
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequence.map(split_input_target) # lambda function call

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(indx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(indx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [8]:
# Creating training batch
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
vocab_size = len(vocab) # 65
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(vocab_size, embedding_dim, rnn_units)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [17]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [18]:
# callback checkpoint
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [21]:
num_epoches = 10

history = model.fit(dataset, epochs=num_epoches, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 10s 55ms/step - loss: 1.2100
Epoch 2/10
172/172 [==============================] - 11s 56ms/step - loss: 1.1772
Epoch 3/10
172/172 [==============================] - 11s 56ms/step - loss: 1.1436
Epoch 4/10
172/172 [==============================] - 11s 57ms/step - loss: 1.1094
Epoch 5/10
172/172 [==============================] - 11s 58ms/step - loss: 1.0742
Epoch 6/10
172/172 [==============================] - 11s 58ms/step - loss: 1.0386
Epoch 7/10
172/172 [==============================] - 11s 58ms/step - loss: 1.0032
Epoch 8/10
172/172 [==============================] - 11s 58ms/step - loss: 0.9671
Epoch 9/10
172/172 [==============================] - 11s 59ms/step - loss: 0.9317
Epoch 10/10
172/172 [==============================] - 11s 60ms/step - loss: 0.8969


In [20]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [22]:
# run model with difference batch size
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
def generate_text(model, start_string):
  num_gen = 1000 # no of char to generate

  # convert our start_string to numbers(vectorizing)
  input_eval = [char2indx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temp = 1.0

  model.reset_states()
  for i in range(num_gen):
    predictions = model(input_eval)
    # remove the batch dimensions
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions/temp
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(indx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))


In [30]:
print(generate_text(model, start_string=u"First Citizen: "))

First Citizen: I will prosper we make
which any other,--you're none of his proje,
Give me another, as I live, like a suard

KING RICHARD II:
I will he, consent then, I talk of my man.

First Soldier:
Foul weary great Aufidius,
Our sea, I'll pine at our hands,
Where I may chance to speak, not better.

Gentleman:
If that am cannot be amended
From off my hands I give a long, to arm the rest,
Are marger: thy fair King Richard's foot:
And all revour, Kate!

SOMERSEM:
At Clarence? 'say this day life, I am as cheap'd of love?

ALONSO:
Thou stept was bit for them
And fall out worthy office, as highness and Harry Pity,
She's not heart:
When let thy consent to pierce arguised?

PAULINA:
Good queen. I have felt three pestilence;
Somedians have any grievy dog that.

GRUMIO:
O Tybalt, welcome.
A fire is yond.

PETRUCHIO:
Believe me, sister, let's have said, LAp--
Sim or her lip aragement
That's left am maiden place; and, whilst Muster Citulent
And send at Henry to your custion.
My fairest crowns us